### Замерим качество Линейной регрессии после обработки данных не просто на отложенной выборке, но и на Кросс-Валидации на 4 фолдах!

In [18]:
import numpy as np
import pandas as pd

processed_data = pd.read_csv('processed_data.csv', index_col='id')

In [19]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,
id2875421,1,930.399753,0,455.0,1.500479
id2377394,0,930.399753,0,663.0,1.807119
id3858529,1,930.399753,0,2124.0,6.392080
id3504673,1,930.399753,0,429.0,1.487155
id2181028,1,930.399753,0,435.0,1.189925


#### ! Не перемешивайте данные

In [20]:
processed_data = processed_data.assign(log_trip_duration=np.log1p(processed_data['trip_duration']))
processed_data = processed_data.drop('trip_duration', axis=1)

X_data = processed_data.drop('log_trip_duration', axis = 1)
y_data = processed_data['log_trip_duration']

processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


In [24]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

selector = KFold(n_splits=4)

### Задание 4
### Замерьте качество (MSLE, как и раньше) на Кросс-валидации, 
### используя MSE от log_trip_duration и назначенный selector

cross_errors = []

for train_index, test_index in selector.split(X=X_data, y=y_data):
    
    X_train, y_train = X_data.values[train_index], y_data.values[train_index]
    X_test, y_test = X_data.values[test_index], y_data.values[test_index]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    error = ((y_test - model.predict(X_test))**2).mean()
    cross_errors.append(error)
    
cross_val_error = np.mean(cross_errors)
print(f"MSLE на Кросс-валидации: {round(cross_val_error, 3)}")

MSLE на Кросс-валидации: 0.426


## Поработал один из хитрых гномов!

В отличие от своих собратьев, третий гном оказался тем еще бездельником в школьные годы, но все равно страстно желал во всем догнать первых двух. И сейчас, желая помочь им в построении модели по предсказанию длительности поездки такси, добавил в данные 20 зашифрованных фичей (их смысл нам не рассказали: какая-то секретная информация о водителях).

Гном думал следующим образом: "Ну не может же модель стать хуже! А тут вот авось и мое нововведение уменьшит ошибку в разы! Тогда и меня станут звать на гномий  data-саммит."

Проверим на кросс-валидации, насколько гном оказался прав!

In [25]:
new_data = pd.read_csv('new_data.csv', index_col='id')

In [26]:
new_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20
id,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,1,1,1,1,1,...,0,0,0,0,0,1.500479,2.251437,3.378234,5.068969,7.605881
id2377394,0,930.399753,0,1.807119,6.498282,0,0,0,0,0,...,0,0,0,0,0,1.807119,3.265681,5.901475,10.664670,19.272331
id3858529,1,930.399753,0,6.392080,7.661527,1,1,1,1,1,...,0,0,0,0,0,6.392080,40.858690,261.172025,1669.432545,10671.146803
id3504673,1,930.399753,0,1.487155,6.063785,1,1,1,1,1,...,0,0,0,0,0,1.487155,2.211629,3.289035,4.891303,7.274125
id2181028,1,930.399753,0,1.189925,6.077642,1,1,1,1,1,...,0,0,0,0,0,1.189925,1.415923,1.684842,2.004837,2.385606


In [30]:
X_data_new = new_data.drop('log_trip_duration', axis = 1)
y_data_new = new_data['log_trip_duration']

In [31]:
### Задание №5
### Замерьте качество (MSLE, как и раньше) на Кросс-валидации, 
### используя MSE от log_trip_duration и назначенный ранее selector

cross_errors = []

selector = KFold(n_splits=4)

for train_index, test_index in selector.split(X=X_data_new, y=y_data_new):
    
    X_train, y_train = X_data_new.values[train_index], y_data_new.values[train_index]
    X_test, y_test = X_data_new.values[test_index], y_data_new.values[test_index]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    error = ((y_test - model.predict(X_test))**2).mean()
    cross_errors.append(error)
    
cross_val_error_2 = np.mean(cross_errors)

print(f"MSLE на Кросс-валидации: {round(cross_val_error_2, 3)}")

MSLE на Кросс-валидации: 140.921


В линейной алгербре зачастую используют понятие **ранга матрицы**. Оно соответствует кол-ву линейно независимых столбцов в матрице. Иными словами, позволяет оценить, есть ли избыток информации в нашем датафрейме. Если ранг матрицы меньше, чем кол-во используемых столбцов, то некоторые фичи следует удалить, ведь иначе возникает ситуация строгой мультиколлинеарности.

Чтобы замерить ранг в наших матрицах объект-признак, можно воспользоваться функцией numpy.linalg.matrix_rank

Константным признаком в данном упражнении можно пренебречь.

In [32]:
### Создайте переменные rank_processed, rank_new 
### Соответственно равные рангу изначальной матрицы
### с данными и рангу матрицы третьего гнома

### Your code is here
np.linalg.matrix_rank(new_data)

5

In [35]:
len(model.coef_)

24

In [42]:
### Создайте переменные num_features_processed, num_features_new
### Соответственно равные кол-ву фичей в изначальной матрицы
### с данными и кол-ву фичей у третьего гнома

### Your code is here
num_features_processed = processed_data.shape[1]
rank_processed = np.linalg.matrix_rank(processed_data)

num_features_new = new_data.shape[1]-1
rank_new = np.linalg.matrix_rank(new_data)

In [43]:
### Задание №6
print(f"В первой модели всего фичей: {num_features_processed}, - а ранг равен {rank_processed}")

print(f"Во второй модели всего фичей: {num_features_new}, - а ранг равен {rank_new}")

В первой модели всего фичей: 5, - а ранг равен 5
Во второй модели всего фичей: 24, - а ранг равен 5


Не кажется ли нам, что из-за новых 20 фичей появилась проблема мультиколлинеарности? Как поступить гному, чтобы, с одной стороны, получить адекватное качество, а с другой стороны, не повредить свое самолюбие и не убирать новые признаки?

Верно! Например, с помощью регуляризации.

Найдите такой параметр регуляризации $\alpha$ для Ridge и Lasso случая, чтобы ошибка MSLE на кросс-валидации оказалась строго меньше 0.4

**ALARM**: используйте процедуру масштабирования данных (воспользуйтесь методом MinMaxScaler) перед тем как применить регуляризацию. Важно - чтобы сохранить концепцию независимости обучения на трейне и на тесте, на каждой итерации кросс-валидации необходимо замерять параметры стандартизации исключительно на трейне, а потом применять на валидационном фолде.

In [44]:
### Пример, как это можно сделать в цикле
### То есть обучить Lasso, учитывая масштабирование
### Исключительно на трейне на каждой итерации



X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso

scores = []

for train_index, test_index in selector.split(X):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    Y_train, Y_test = Y.values[train_index], Y.values[test_index]
    
    ### Фитим исключительно на трейне!
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    ### Применяем обученный scaler на трейн и тест
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    ### max_iter иногда требуется ставить побольше, 
    ### особенно когда данных много и/или они сложные
    ### этот параметр регулирует верхнюю границу кол-ва
    ### итераций во время обучения
    ### подробнее - в документации
    
    ### По дефолту здесь параметр регуляризации alpha=1
    
    model_lasso = Lasso(max_iter=100000) 
    model_lasso.fit(X_train_scaled, Y_train)
    
    predictions = model_lasso.predict(X_test_scaled)
    
    scores.append(np.mean((predictions - Y_test)**2))

    
print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")

### P.S. если вы уже умеете работать с конструкциями
### Pipeline, GridSearchCV, cross_validate
### Можете использовать их. Мы познакомимся с ними позже.

MSLE на Кросс-валидации равен: 0.6332330617999488


In [66]:
### Теперь найдите оптимальный параметр регуляризации
### для случая Ridge
### Напомним: ошибка на кросс-валидации должно быть 
### строго меньше 0.4

### Задание №7
### Пример, как это можно сделать в цикле
### То есть обучить Lasso, учитывая масштабирование
### Исключительно на трейне на каждой итерации



X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso


alphas = np.linspace(0.00001, 0.0001, num=30)

for alpha in alphas:
    scores = []
    for train_index, test_index in selector.split(X):

        X_train, X_test = X.values[train_index], X.values[test_index]
        Y_train, Y_test = Y.values[train_index], Y.values[test_index]

        ### Фитим исключительно на трейне!
        scaler = MinMaxScaler()
        scaler.fit(X_train)

        ### Применяем обученный scaler на трейн и тест
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        ### max_iter иногда требуется ставить побольше, 
        ### особенно когда данных много и/или они сложные
        ### этот параметр регулирует верхнюю границу кол-ва
        ### итераций во время обучения
        ### подробнее - в документации

        ### По дефолту здесь параметр регуляризации alpha=1

        model_lasso = Lasso(alpha=alpha, max_iter=100000) 
        model_lasso.fit(X_train_scaled, Y_train)

        predictions = model_lasso.predict(X_test_scaled)

        scores.append(np.mean((predictions - Y_test)**2))
        
    print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")
    print(f"alpha на Кросс-валидации равен: {alpha}")
    print()


MSLE на Кросс-валидации равен: 0.3984816859563724
alpha на Кросс-валидации равен: 1e-05

MSLE на Кросс-валидации равен: 0.38988994594712667
alpha на Кросс-валидации равен: 1.310344827586207e-05

MSLE на Кросс-валидации равен: 0.39519988554125385
alpha на Кросс-валидации равен: 1.6206896551724137e-05

MSLE на Кросс-валидации равен: 0.40057995786635775
alpha на Кросс-валидации равен: 1.9310344827586207e-05



KeyboardInterrupt: 

In [69]:
### Найдите оптимальный параметр регуляризации
### для случая Lasso
### Напомним: ошибка на кросс-валидации должно быть 
### строго меньше 0.4

### Задание №8
### Your code is here
X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge


alphas = np.linspace(5.2068965517241384e-05, 5.241379310344828e-05, num=10)

for alpha in alphas:
    scores = []
    for train_index, test_index in selector.split(X):

        X_train, X_test = X.values[train_index], X.values[test_index]
        Y_train, Y_test = Y.values[train_index], Y.values[test_index]

        ### Фитим исключительно на трейне!
        scaler = MinMaxScaler()
        scaler.fit(X_train)

        ### Применяем обученный scaler на трейн и тест
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        ### max_iter иногда требуется ставить побольше, 
        ### особенно когда данных много и/или они сложные
        ### этот параметр регулирует верхнюю границу кол-ва
        ### итераций во время обучения
        ### подробнее - в документации

        ### По дефолту здесь параметр регуляризации alpha=1

        model_lasso = Ridge(alpha=alpha, max_iter=100000) 
        model_lasso.fit(X_train_scaled, Y_train)

        predictions = model_lasso.predict(X_test_scaled)

        scores.append(np.mean((predictions - Y_test)**2))
        
    print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")
    print(f"alpha на Кросс-валидации равен: {alpha}")
    print()



MSLE на Кросс-валидации равен: 0.4075753211958122
alpha на Кросс-валидации равен: 5.03448275862069e-05

MSLE на Кросс-валидации равен: 0.4055395045834771
alpha на Кросс-валидации равен: 5.0689655172413794e-05

MSLE на Кросс-валидации равен: 0.4039608910607571
alpha на Кросс-валидации равен: 5.103448275862069e-05

MSLE на Кросс-валидации равен: 0.40282429829272226
alpha на Кросс-валидации равен: 5.1379310344827586e-05

MSLE на Кросс-валидации равен: 0.40211503929347003
alpha на Кросс-валидации равен: 5.172413793103448e-05

MSLE на Кросс-валидации равен: 0.4018189046402733
alpha на Кросс-валидации равен: 5.2068965517241384e-05

MSLE на Кросс-валидации равен: 0.40192214541127513
alpha на Кросс-валидации равен: 5.241379310344828e-05

MSLE на Кросс-валидации равен: 0.40241145679413354
alpha на Кросс-валидации равен: 5.2758620689655176e-05

MSLE на Кросс-валидации равен: 0.4032739623537913
alpha на Кросс-валидации равен: 5.310344827586207e-05

MSLE на Кросс-валидации равен: 0.404497198919930